# CUDA SEMANTICS
`torch.cuda`用于部署以及运行 CUDA 操作，它会对目前所选的 GPU 进行追踪，所有 CUDA 张量均会在所被分配的相应设备上生成，相关操作的结果也会被部署至相同的设备上；所分配的设备可在`torch.cuda.device`上下文管理器来进行更改；

除通过`copy_()`、`to()`、`cuda()`等操作，以及启用点对点内存访问时，一般不支持跨 GPU 的操作；

Below you can find a small example showcasing this:

```python
cuda = torch.device('cuda')     # Default CUDA device
cuda0 = torch.device('cuda:0')
cuda2 = torch.device('cuda:2')  # GPU 2 (these are 0-indexed)

x = torch.tensor([1., 2.], device=cuda0)
# x.device is device(type='cuda', index=0)
y = torch.tensor([1., 2.]).cuda()
# y.device is device(type='cuda', index=0)

with torch.cuda.device(1):
    # allocates a tensor on GPU 1
    a = torch.tensor([1., 2.], device=cuda)

    # transfers a tensor from CPU to GPU 1
    b = torch.tensor([1., 2.]).cuda()
    # a.device and b.device are device(type='cuda', index=1)

    # You can also use ``Tensor.to`` to transfer a tensor:
    b2 = torch.tensor([1., 2.]).to(device=cuda)
    # b.device and b2.device are device(type='cuda', index=1)

    c = a + b
    # c.device is device(type='cuda', index=1)

    z = x + y
    # z.device is device(type='cuda', index=0)

    # even within a context, you can specify the device
    # (or give a GPU index to the .cuda call)
    d = torch.randn(2, device=cuda2)
    e = torch.randn(2).to(cuda2)
    f = torch.randn(2).cuda(cuda2)
    # d.device, e.device, and f.device are all device(type='cuda', index=2)
```




## TensorFloat-32(TF32) on Ampere devices
PyTorch 1.7 及以上版本中有一个`allow_tf32`标识符，其默认值为 True，其志控制 PyTorch 是否使用 TensorFloat32 张量核，该机制通过将输入数据四舍五入入至 10 位尾数，以 32 位浮点精度对结果汇集 (accumulate)，并利用`torch.float32`张量来提高矩阵乘法和卷积的性能；
```python
assert torch.backends.cuda.matmul.allow_tf32 is True
assert torch.backends.cudnn.allow_tf32 is True
```
该机制对计算的加速可参见下面的示例：
```python
def matmul_time(a, b):
    t0 = time.time()
    x = a @ b
    return time.time() - t0

a_FP64 = torch.randn(10240, 10240, dtype=torch.double, device='cuda')
b_FP64 = torch.randn(10240, 10240, dtype=torch.double, device='cuda')

print(matmul_time(a_FP64, b_FP64))  # => 1.05986
print(matmul_time(a_FP64.float(), b_FP64.float()))  # => 0.00143
```

更多有关 TF32 的信息参见 NVIDIA 博客及官网：
- [TensorFloat-32](https://blogs.nvidia.com/blog/2020/05/14/tensorfloat-32-precision-format/)
- [CUDA 11](https://developer.nvidia.com/blog/cuda-11-features-revealed/)
- [Ampere architecture](https://developer.nvidia.com/blog/nvidia-ampere-architecture-in-depth/)




## Asynchronous execution
默认情况下 GPU 操作是异步进行的；在调用使用 GPU 的函数时，pytorch 会将这些操作进行排队并分配到特定的设备上，以并行地执行而非顺序执行，进而可以同步地执行更多 CPU 和其他 GPU 上的操作；通常情况下函数调用者是无法观察到异步计算的，因为每个设备按照分配给其的操作的队列顺序对这些操作进行计算，同时 Pytorch 在 CPU 和 GPU 以及不同 GPU 之间复制数据时会自动的执行一些同步操作，进而计算执行时看似是同步的；

可以通过设置环境变量`CUDA_LAUNCH_BLOCKING=1`强制同步计算；这样在 GPU 出现错误时可以及时发现，因为在异步执行时，这些错误会在操作执行后才会抛出，进而堆栈追踪 (stack track) 无法显示出错的请求位置；

异步计算的一个缺陷是，其对时间的测量是不准确的；若需要获得精确的测量，应先调用`torch.cuda.synchronize()`或像下面的示例中使用`torch.cuda.Event`来记录时间：
```python
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)
start_event.record()
# Run some things ...
end_event.record()
torch.cuda.synchronize()  # Wait for the events to be recorded!
elapsed_time_ms = start_event.elapsed_time(end_event)
```
一种例外是一些函数`to()`和`copy_()`接收一个`non_blocking`参数，该参数可以在不必要时候绕过同步的过程；另一种例外是 CUDA streams：



### CUDA streams
CUDA 流是一个属于特定设备的线性执行序列；默认情况下，每个设备使用自己的默认流；每个流中的操作会按其创建的顺序序列化，但不同流的操作可以以任何相对顺序并发执行 (除使用同步函数如`synchronize()`或`wait_stream()`等以外)；例如下面的代码可能会出现错误：
```python
cuda = torch.device('cuda')
s = torch.cuda.Stream()  # Create a new stream.
A = torch.empty((100, 100), device=cuda).normal_(0.0, 1.0)
with torch.cuda.stream(s):
    # sum() may start execution before normal_() finishes!
    B = torch.sum(A)
```
如上所述，在当前的流是默认流时，PyTorch 在移动数据时会自动执行一些同步操作；然而如果当前流不是默认流，则需要人工执行同步的操作；



### Stream semantics of backward passes
每个反向 CUDA 操作与其相应的前向操作会在一个流上被执行；在人工提供反向传播时初始梯度的 CUDA 张量如`autograd.backward(..., grad_tensors=initial_grads)`、`autograd.grad(..., grad_outputs=initial_grads)`、`tensor.backward(..., gradient=initial_grad)`等时，添加初始梯度数据和调用向后传递的操作与任何一对操作具有相同的流语义 (stream-semantics) 关系：
```python
# populate initial_grad and invoke backward in the same stream context
# which is safe
with torch.cuda.stream(strm):
    loss.backward(gradient=torch.ones_like(loss))

# populate initial_grad and invoke backward in different stream contexts
# unsafe without synchronization
initial_grad = torch.ones_like(loss)
with torch.cuda.stream(strm):
    loss.backward(gradient=initial_grad)

# safe with synchronization
initial_grad = torch.ones_like(loss)
strm.wait_stream(torch.cuda.current_stream())
with torch.cuda.stream(strm):
    initial_grad.record_stream(strm)
    loss.backward(gradient=initial_grad)
```





## Memory management
PyTorch 使用缓存内存分配器来加速内存分配，进而可以在不进行设备同步的情况下快速回收内存；然而分配器管理未使用的内存在`nvidia-smi`中显示仍为占用状态；可以通过使用`memory_allocated()`和`max_memory_allocated()`来监测张量所占用的内存，以及利用`memory_reserved()`和`max_memory_reserved()`来监测缓存分配器所管理的总内存；通过调用`empty_cache()`可以从 PyTorch 释放所有未使用的缓存内存以便其他 GPU 程序使用；由于被张量占用的 GPU 内存不会释放，因此该函数不能增加可供 PyTorch 使用的 GPU 内存数量；

此外，利用`memory_stats()`可以进行更全面的内存基准测试 (benchmarking)，通过`memory_snapshot()`可以捕获内存分配器的完整状态，进而有助于理解由代码生成的底层分配模式；缓存分配器的使用会干扰内存检查工具如`cuda-memcheck`，若需要使用`cuda-memcheck`调试内存错误，应在环境中设置`PYTORCH_NO_CUDA_MEMORY_CACHING=1`以禁用缓存




## cuFFT plan cache
For each CUDA device, an LRU cache of cuFFT plans is used to speed up repeatedly running FFT methods (e.g., torch.fft.fft()) on CUDA tensors of same geometry with same configuration. Because some cuFFT plans may allocate GPU memory, these caches have a maximum capacity.

You may control and query the properties of the cache of current device with the following APIs:

torch.backends.cuda.cufft_plan_cache.max_size gives the capacity of the cache (default is 4096 on CUDA 10 and newer, and 1023 on older CUDA versions). Setting this value directly modifies the capacity.

torch.backends.cuda.cufft_plan_cache.size gives the number of plans currently residing in the cache.

torch.backends.cuda.cufft_plan_cache.clear() clears the cache.

To control and query plan caches of a non-default device, you can index the torch.backends.cuda.cufft_plan_cache object with either a torch.device object or a device index, and access one of the above attributes. E.g., to set the capacity of the cache for device 1, one can write torch.backends.cuda.cufft_plan_cache[1].max_size = 10.





## Best practices
### Device-agnostic code
由于 PyTorch 的设计结构要求，使用者需要显式地编写跨设备的代码；例如创建一个张量作为 RNN 的初始隐藏状态时，首先要确定 GPU 是否可用以及所使用的设备，随后可以直接在相应设备上创建张量，或利用`to()`、`cuda()`函数进行转换；
```python
if not args.disable_cuda and torch.cuda.is_available():
    args.device = torch.device('cuda')
else:
    args.device = torch.device('cpu')

# create tensors on device
x = torch.empty((8, 42), device=args.device)
net = Network().to(device=args.device)

# use .to() method
for i, x in enumerate(train_loader):
    x = x.to(args.device)
```
当在一个系统上使用多 GPU 时，可以使用`CUDA_VISIBLE_DEVICES`标识符来管理 PyTorch 可以使用哪些 GPU；如上所述，用户需要手动控制张量在哪个 GPU 上创建，最佳方式是使用`torch.cuda.device`上下文管理器；
```python
print("Outside device is 0")
with torch.cuda.device(1):
    print("Inside device is 1")
print("Outside device is still 0")
```
若需要创建一个和已有张量具有相同类型、位于相同设备上的张量，可以利用`torch.Tensor.new_x`方法；对于前向传播过程中需要创建张量的模型，这是最好的方式；若需要创建一个与已有张量具有相同类型和形状的全 0 或全 1 张量，可以利用`ones_like()`或`zeros_like()`实现，这两个函数也会保留`Tensor`的`torch.device`和`torch.dtype`；
```python
x_cpu = torch.empty(2, 3)
x_gpu = torch.empty(2, 3, device="cuda")
y_cpu = torch.ones_like(x_cpu)
y_gpu = torch.zeros_like(x_gpu)
assert x_cpu.device == y_cpu.device and x_cpu.dtype == y_cpu.dtype
assert x_gpu.device == y_gpu.device and x_gpu.dtype == y_gpu.dtype
```



### Use pinned memory buffers
当主机到 GPU 的拷贝文件来源于固定内存时，对数据的读取会更快；CPU 张量和存储均暴露了一个`pin_memory()`方法，该方法可以返回存储在固定区域中的对象的副本；

此外，在固定张量或存储后便可使用异步 GPU 副本——只需将`non_blocking=True`参数传递给`to()`或`cuda()`，进而可以式含有计算的数据转换之间进行重叠；

通过将`pin_memory=True`传递`DataLoader`的构造函数，进而可以使其返回放置在固定内存中的 batch；



### Use `nn.parallel.DistributedDataParallel`
大多涉及批输入和多 GPU 的使用应该默认使用`DistributedDataParallel`；在多进程处理中使用 CUDA 模型有很多重要的注意事项，如果没有准确地满足数据处理要求，那么程序很可能会有不正确或未定义的操作；

通常建议使用`DistributedDataParallel`而非`DataParallel`来进行多 GPU 训练，即便只有一个节点；`DistributedDataParallel`和`DataParallel`区别在于，前者为每个 GPU 创建一个进程，而后者使用多线程。多进程机制下每个 GPU 都有自己的专用进程，进而避免了给 Python 解释器 GIL 带来的性能开销；使用`DistributedDataParallel`时可以利用`torch.distributedlaunch`实用程序来启动程序，请参阅第三方 [backend](https://pytorch.org/docs/stable/distributed.html#distributed-launch)

# Distributed Data Parallel
`torch.nn.parallel.DistributedDataParallel` (DDP) 可以执行分布式的数据并行训练，下面描述其工作机制以及实现细节；注意！`DistributedDataParallel`的实现会随着代码更新而不断改变，以下教程是基于 PyTorch 1.4 编写的

## Example
下面的示例使用`torch.nn.Linear`作为本地模型并用 DDP 对其进行包装，再在 DDP 模型进行一个前向传递、一个后向传递和一个优化步骤；在此之后本地模型的参数会被更新，不同进程上的模型应该完全相同；
```python
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP

def example(rank, world_size):
    # create default process group
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

    model = nn.Linear(10, 10).to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    outputs = ddp_model(torch.randn(20, 10).to(rank))
    labels = torch.randn(20, 10).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()

def main():
    world_size = 2
    mp.spawn(example, args=(world_size,), nprocs=world_size, join=True)

if __name__=="__main__":
    main()
```

## Internal Design
本节通过深入单次迭代中每个步骤的细节，揭示`DistributedDataParallel`框架是如何工作的；

- **Prerequisite**: DDP 依赖 c10d `ProcessGroup`进行信息交流，进而程序应在构造 DDP 之前创建`ProcessGroup`实例；
- **Construction**: DDP 构造函数会接受一个本地模块的引用，从级别 0 的进程利用广播机制将`state_dict()`传递给给组中的所有其他进程，以确保所有模型副本都从完全相同的`state_dict`启动；之后每个 DDP 进程创建一个本地的`Reducer`，其负责随后的反向传递过程中梯度的同步；为了提高信息交流的效率，`Reducer`会将参数梯度组织成储存桶，并每次减少一个储存桶；储存桶大小可以通过设置 DDP 构造函数的`bucket_cap_mb`参数来配置；在构造 DDP 时会根据储存桶和参数大小对参数梯度到储存桶的映射进行确定；模型参数按照给定模型`Model.parameters()`相反的顺序大致分配到存储桶中，使用倒序可以使参数的梯度在反向传播过程中以近似的顺序准备好，例如下面的示例中，`grad0`和`grad1`在`bucket1`中，其他两个梯度在`bucket0`中；当然这个假设未必总是正确的，当该假设不成立时，由于`Reducer`不能及时启动通信，进而可能会损害 DDP 的反向传播速度；除使用储存桶外，`Reducer`在构造时还会自动为每个参数注册 hook，这些 hook 会在反向传播中被触发；
- **Forward Pass**: DDP 首先接受输入并将其传递给本地模型，如果`find_unused_parameters=True`则对本地模型输出进行分析；该模式允许在模型的子图上进行反向传播，DDP 从模型输出遍历 aotugrad 图，将所有未使用的参数进行标记，进而找到反向传播过程需要涉及到的参数；反向传播过程中`Reducer`只等待未准备就绪的参数，但它仍然会减少所有的储存桶；目前而言，将参数梯度标记为就绪的状态并不能帮助 DDP 跳过储存桶，但可以防止 DDP 在反向传播过程中一直等待所缺失的梯度；需要注意的是，遍历 aotugrad 图会增加额外开销，因而程序最好在必要时再设置`find_unused_parameters=True`；
**Backward Pass**: 反向传播中 DDP 使用在构造时注册的 autograd hook 来触发梯度同步；当一个梯度准备就绪后，其相应的梯度累加器上的 DDP hook 便会启动，随后 DDP 将该参数梯度标记为要减小的状态，一个存储桶中的梯度都准备好后，`Reducer`在该存储桶上启动一个异步的`allreduce`来计算所有进程的梯度的平均值；当所有储存桶都准备好后，`Reducer`将组织程序进一步运行以等待所有的`allreduce`完成；操作完成会将平均梯度写入所有参数的`param.grad`字段；进而反向传播后不同 DDP 进程中相同参数上的`grad`字段应该是相同的；
**Optimizer Step**: 从优化器的角度来看，其正在优化一个局部模型；由于所有 DDP 进程上的模型副本都从相同的状态开始并在每次迭代中具有相同的平均梯度，进而其可以保持同步；

![ddp_grad_sync.png](https://user-images.githubusercontent.com/16999635/72401724-d296d880-371a-11ea-90ab-737f86543df9.png)

DDP 要求所有进程上的`Reducer`实例以完全相同的顺序调用`allreduce`，即总是按照储存桶索引顺序而非实际储存桶就绪顺序运行`allreduce`；跨进程不匹配的`allreduce`顺序可能导致错误的结果或 DDP 反向传播过程中断


## Implementation
Below are pointers to the DDP implementation components. The stacked graph shows the structure of the code.

### ProcessGroup
- `ProcessGroup.hpp`: 其包含了所有进程组实现的抽象 API；c10d 库提供了 3 种即用的实现，即`ProcessGroupGloo`、`ProcessGroupNCCL`、`ProcessGroupMPI`；`DistributedDataParallel`初始化时使用`ProcessGroup::broadcast()`将 0 等级进程的模型 state 发送给其他进程，使用`ProcessGroup::allreduce()`对梯度进行加和；
- `Store.hpp`: 协助 rendezvous 服务以让流程组实例找到彼此


### DistributedDataParallel
- `distributed.py`: 是 DDP 的 Python 入口点 (entry point)，它实现了调用 C++ 库的`DistributedDataParallel`模块的初始化和`forward`函数；当一个 DDP 进程在多个设备上工作时，该文件的`_sync_param`函数会执行进程内的参数同步操作；此外，该文件还从等级 0 的进程将模型广播至所有其他进程；进程间参数的同步在`Reducer.cpp`中进行；
- `comm.h`: 实现联合广播的帮助函数，该函数在初始化期间被调用以对模型的 state 进行广播，并在前向传播之前对模型的缓冲区进行缓冲
- `reducer.h`: 提供反向传递过程中梯度同步的核心实现，其含有 3 个 entry point 函数：
    - `Reducer`: 其构造函数会在`distribute.py`中进行调用，并将`Reducer::autograd_hook()`注册到梯度累加器中；
    - `autograd_hook()` 函数会在所有梯度张量准备好时由自动求导函数调用；
    - `prepare_for_backward()`会在 DDP 前向传播后进行调用；当给 DDP 构造函数传递的参数`find_unused_parameters`为 True 时，该函数会遍历自动求导图以查找未使用的参数；

<img src="https://user-images.githubusercontent.com/16999635/72313120-4e7c1c80-3658-11ea-9c6d-44336b2daeac.png" width=360>

#  

#  

## PyTorch Distributed Overview

### Introduction
从 PyTorch v1.6.0 开始，`torch.distributed`中的功能可以分为三个主要组件：
- 分布式数据并行训练（DDP）是一种广泛采用的单程序多数据训练范例；利用 DDP 可以在每个进程上复制模型，每个模型副本会获得一组互不相同的输入数据；DDP 负责不同设备之间梯度张量的同步，并通过使其与梯度计算重叠进而加快训练速度；
- 基于 RPC 的分布式训练用于支持无法适应数据并行训练的常规训练结构，例如分布式管道并行、参数服务器范式、DDP 与其他训练范式的组合等；其有助于管理远程对象的运行生命周期，并将自动求导机制扩展到机器范围之外；
- ？？？？？？？？？？？？？集体通信（c10d）库支持跨组内的进程发送张量。 它提供了集体通信 API（例如all_reduce和all_gather）和 P2P 通信 API（例如send和 isend）。 从 v1.6.0 开始，DDP 和 RPC（ProcessGroup 后端）建立在 c10d 上，其中前者使用集体通信，而后者使用 P2P 通信。 通常，开发人员无需直接使用此原始通信 API，因为上述 DDP 和 RPC 功能可以满足许多分布式训练方案的需求。 但是，在某些情况下，此 API 仍然很有帮助。 一个示例是分布式参数平均，其中应用希望在反向传播之后计算所有模型参数的平均值，而不是使用 DDP 来传递梯度。 这可以使通信与计算脱钩，并允许对通信内容进行更细粒度的控制，但另一方面，它也放弃了 DDP 提供的性能优化。 用 PyTorch 编写分布式应用显示了使用 c10d 通信 API 的示例。

现有的大多数文档都是为 DDP 或 RPC 编写的，下面将详细介绍这两个组件；

### Data Parallel Training
PyTorch 为数据并行训练提供了多种可选方案；那些从简单到复杂、从 prototype 到生产的应用的共同的发展轨迹是：
- 如果一个 GPU 便可容纳训练模型和数据，并且不关心训练速度时，可以使用单设备训练；
- 如果服务器上有多个 GPU 并希望以最少的代码修改量来加快训练速度，可以使用单机器多 GPU 数据并行 API [DataParallel](https://pytorch.org/docs/master/generated/torch.nn.DataParallel.html) 训练；
- 如果想进一步加快训练速度并有意编写更多代码来进行配置，可以使用单机多 GPU 分布式数据并行 API [ DistributedDataParallel](https://pytorch.org/docs/master/generated/torch.nn.parallel.DistributedDataParallel.html) 进行训练；
- 如果应用需要跨计算机边界进行 scale，可以使用多计算机分布式数据并行和启动脚本；
- 如果预计会出现错误（如 OOM）或者在训练过程中资源可以动态加入和离开，可使用 torchelastic 启动分布式训练；

需要说明的是，数据并行训练还可以与自动混合精度（AMP）一起使用；

### nn.DataParallel
DataParallel 对单机器多 GPU 并行处理进行了实现，其只需更改一行代码即可实现，具体参看教程[Data Parallelism](https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html)；需要注意的是，尽管 DataParallel 易于使用，但其通常无法提供最佳性能，因为 DataParallel 实现时会在每个正向传播中复制模型，并且其单进程多线程并行性自然会遭受 GIL 争用；为了获得更好的性能可使用 DistributedDataParallel；

### nn.parallel.DistributedDataParallel
与 DataParallel 相比，DistributedDataParallel 还需要设置调用`init_process_group`；DDP 使用多进程并行性，因此在模型副本之间没有 GIL 争用；此外，该模型是在 DDP 构建时而不是在每个正向传播时进行广播的，进而有助于加快训练速度；DDP 附带了几种性能优化技术；详细说明请参阅 [DDP 论文](http://www.vldb.org/pvldb/vol13/p3005-li.pdf)(VLDB'20)；

DDP 相关材料如下：

DDP 注解提供了一个入门示例，并简要介绍了其设计和实现。 如果这是您第一次使用 DDP，请从本文档开始。
分布式数据并行入门解释了 DDP 训练的一些常见问题，包括不平衡的工作量，检查点和多设备模型。 请注意，DDP 可以轻松与单机模型并行最佳实践教程中描述的单机多设备模型并行性结合。
启动和配置分布式数据并行应用文档显示了如何使用 DDP 启动脚本。
使用 Amazon AWS 的 PyTorch 分布式训练器演示了如何在 AWS 上使用 DDP。
TorchElastic
随着应用复杂性和规模的增长，故障恢复成为当务之急。 有时，使用 DDP 时不可避免地会遇到 OOM 之类的错误，但是 DDP 本身无法从这些错误中恢复，基本的try-except块也无法工作。 这是因为 DDP 要求所有进程以紧密同步的方式运行，并且在不同进程中启动的所有AllReduce通信都必须匹配。 如果组中的某个进程抛出 OOM 异常，则很可能导致不同步（AllReduce操作不匹配），从而导致崩溃或挂起。 如果您期望在训练过程中发生故障，或者资源可能会动态离开并加入，请使用 Torrlastic 启动分布式数据并行训练。

通用分布式训练
许多训练范式不适合数据并行性，例如参数服务器范式，分布式管道并行性，具有多个观察者或智能体的强化学习应用等。 torch.distributed.rpc旨在支持一般的分布式训练方案 。

torch.distributed.rpc包具有四个主要支柱：

RPC 支持在远程工作器上运行给定函数
RRef 帮助管理远程对象的生存期。 引用计数协议在 RRef 注解中提供。
分布式自动微分将自动微分引擎扩展到机器范围之外。 有关更多详细信息，请参考分布式 Autograd 设计。
分布式优化器，它使用分布式 Autograd 引擎计算的梯度自动与所有参与的工作器联系以更新参数。
RPC 教程如下：

分布式 RPC 框架入门教程首先使用一个简单的强化学习（RL）示例来演示 RPC 和 RRef。 然后，它对 RNN 示例应用了基本的分布式模型并行性，以展示如何使用分布式 Autograd 和分布式优化器。
使用分布式 RPC 框架实现参数服务器教程借鉴了 HogWild 的训练精神，并将其应用于异步参数服务器（PS）训练应用。
使用 RPC 的分布式管道并行化教程将单机管道并行示例（在单机模型并行最佳实践中介绍）扩展到了分布式环境，并展示了如何使用 RPC 来实现它 。
使用异步执行实现批量 RPC 教程演示了如何使用@rpc.functions.async_execution装饰器实现 RPC 批量。这可以帮助加速推理和训练。 它使用了以上教程 1 和 2 中采用的类似 RL 和 PS 示例。
将分布式DataParallel与分布式 RPC 框架结合教程演示了如何将 DDP 与 RPC 结合使用分布式数据并行性和分布式模型并行性来训练模型。


## 单机模型并行最佳实践

模型并行在分布式训练技术中被广泛使用。 先前的帖子已经解释了如何使用DataParallel在多个 GPU 上训练神经网络； 此功能将相同的模型复制到所有 GPU，其中每个 GPU 消耗输入数据的不同分区。 尽管它可以极大地加快训练过程，但不适用于模型太大而无法容纳单个 GPU 的某些用例。 这篇文章展示了如何通过使用模型并行解决该问题，与DataParallel相比，该模型将单个模型拆分到不同的 GPU 上，而不是在每个 GPU 上复制整个模型（具体来说， 假设模型m包含 10 层：使用DataParallel时，每个 GPU 都具有这 10 层中的每一个的副本，而当在两个 GPU 上并行使用模型时，每个 GPU 可以承载 5 层。

模型并行化的高级思想是将模型的不同子网放置在不同的设备上，并相应地实现forward方法以在设备之间移动中间输出。 由于模型的一部分仅在任何单个设备上运行，因此一组设备可以共同为更大的模型服务。 在本文中，我们将不会尝试构建庞大的模型并将其压缩到有限数量的 GPU 中。 取而代之的是，本文着重展示并行模型的思想。 读者可以将这些想法应用到实际应用中。

注意

对于模型跨越多个服务器的分布式模型并行训练，请参考分布式 RPC 框架入门，以获取示例和详细信息。

基本用法
让我们从包含两个线性层的玩具模型开始。 要在两个 GPU 上运行该模型，只需将每个线性层放置在不同的 GPU 上，然后移动输入和中间输出以匹配层设备。

import torch
import torch.nn as nn
import torch.optim as optim

class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = torch.nn.Linear(10, 10).to('cuda:0')
        self.relu = torch.nn.ReLU()
        self.net2 = torch.nn.Linear(10, 5).to('cuda:1')

    def forward(self, x):
        x = self.relu(self.net1(x.to('cuda:0')))
        return self.net2(x.to('cuda:1'))Copy
请注意，除了五个to(device)调用将线性层和张量放置在适当的设备上之外，上述ToyModel看起来非常类似于在单个 GPU 上实现它的方式。 那是模型中唯一需要更改的地方。 backward()和torch.optim将自动处理梯度，就像模型在一个 GPU 上一样。 调用损失函数时，只需确保标签与输出位于同一设备上。

model = ToyModel()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

optimizer.zero_grad()
outputs = model(torch.randn(20, 10))
labels = torch.randn(20, 5).to('cuda:1')
loss_fn(outputs, labels).backward()
optimizer.step()Copy
将模型并行应用于现有模块
只需进行几行更改，就可以在多个 GPU 上运行现有的单 GPU 模块。 以下代码显示了如何将torchvision.models.resnet50()分解为两个 GPU。 这个想法是继承现有的ResNet模块，并在构建过程中将层拆分为两个 GPU。 然后，通过相应地移动中间输出，覆盖forward方法来缝合两个子网。

from torchvision.models.resnet import ResNet, Bottleneck

num_classes = 1000

class ModelParallelResNet50(ResNet):
    def __init__(self, *args, **kwargs):
        super(ModelParallelResNet50, self).__init__(
            Bottleneck, [3, 4, 6, 3], num_classes=num_classes, *args, **kwargs)

        self.seq1 = nn.Sequential(
            self.conv1,
            self.bn1,
            self.relu,
            self.maxpool,

            self.layer1,
            self.layer2
        ).to('cuda:0')

        self.seq2 = nn.Sequential(
            self.layer3,
            self.layer4,
            self.avgpool,
        ).to('cuda:1')

        self.fc.to('cuda:1')

    def forward(self, x):
        x = self.seq2(self.seq1(x).to('cuda:1'))
        return self.fc(x.view(x.size(0), -1))Copy
对于模型太大而无法放入单个 GPU 的情况，上述实现解决了该问题。 但是，您可能已经注意到，如果模型合适，它将比在单个 GPU 上运行它要慢。 这是因为在任何时间点，两个 GPU 中只有一个在工作，而另一个在那儿什么也没做。 由于中间输出需要在layer2和layer3之间从cuda:0复制到cuda:1，因此性能进一步恶化。

让我们进行实验以更定量地了解执行时间。 在此实验中，我们通过运行随机输入和标签来训练ModelParallelResNet50和现有的torchvision.models.resnet50()。 训练后，模型将不会产生任何有用的预测，但是我们可以对执行时间有一个合理的了解。

import torchvision.models as models

num_batches = 3
batch_size = 120
image_w = 128
image_h = 128

def train(model):
    model.train(True)
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    one_hot_indices = torch.LongTensor(batch_size) \
                           .random_(0, num_classes) \
                           .view(batch_size, 1)

    for _ in range(num_batches):
        # generate random inputs and labels
        inputs = torch.randn(batch_size, 3, image_w, image_h)
        labels = torch.zeros(batch_size, num_classes) \
                      .scatter_(1, one_hot_indices, 1)

        # run forward pass
        optimizer.zero_grad()
        outputs = model(inputs.to('cuda:0'))

        # run backward pass
        labels = labels.to(outputs.device)
        loss_fn(outputs, labels).backward()
        optimizer.step()Copy
上面的train(model)方法使用nn.MSELoss作为损失函数，并使用optim.SGD作为优化器。 它模拟了对128 X 128图像的训练，这些图像分为 3 批，每批包含 120 张图像。 然后，我们使用timeit来运行train(model)方法 10 次，并绘制带有标准差的执行时间。

import matplotlib.pyplot as plt
plt.switch_backend('Agg')
import numpy as np
import timeit

num_repeat = 10

stmt = "train(model)"

setup = "model = ModelParallelResNet50()"
# globals arg is only available in Python 3\. In Python 2, use the following
# import __builtin__
# __builtin__.__dict__.update(locals())
mp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
mp_mean, mp_std = np.mean(mp_run_times), np.std(mp_run_times)

setup = "import torchvision.models as models;" + \
        "model = models.resnet50(num_classes=num_classes).to('cuda:0')"
rn_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
rn_mean, rn_std = np.mean(rn_run_times), np.std(rn_run_times)

def plot(means, stds, labels, fig_name):
    fig, ax = plt.subplots()
    ax.bar(np.arange(len(means)), means, yerr=stds,
           align='center', alpha=0.5, ecolor='red', capsize=10, width=0.6)
    ax.set_ylabel('ResNet50 Execution Time (Second)')
    ax.set_xticks(np.arange(len(means)))
    ax.set_xticklabels(labels)
    ax.yaxis.grid(True)
    plt.tight_layout()
    plt.savefig(fig_name)
    plt.close(fig)

plot([mp_mean, rn_mean],
     [mp_std, rn_std],
     ['Model Parallel', 'Single GPU'],
     'mp_vs_rn.png')Copy


结果表明，模型并行实现的执行时间比现有的单 GPU 实现长4.02/3.75-1=7%。 因此，我们可以得出结论，在 GPU 之间来回复制张量大约有 7% 的开销。 有待改进的地方，因为我们知道两个 GPU 之一在整个执行过程中处于空闲状态。 一种选择是将每个批量进一步划分为拆分流水线，以便当一个拆分到达第二子网时，可以将下一个拆分馈入第一子网。 这样，两个连续的拆分可以在两个 GPU 上同时运行。

通过流水线输入加快速度
在以下实验中，我们将每个 120 图像批量进一步分为 20 图像分割。 当 PyTorch 异步启动 CUDA 操作时，该实现无需生成多个线程即可实现并发。

class PipelineParallelResNet50(ModelParallelResNet50):
    def __init__(self, split_size=20, *args, **kwargs):
        super(PipelineParallelResNet50, self).__init__(*args, **kwargs)
        self.split_size = split_size

    def forward(self, x):
        splits = iter(x.split(self.split_size, dim=0))
        s_next = next(splits)
        s_prev = self.seq1(s_next).to('cuda:1')
        ret = []

        for s_next in splits:
            # A. s_prev runs on cuda:1
            s_prev = self.seq2(s_prev)
            ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

            # B. s_next runs on cuda:0, which can run concurrently with A
            s_prev = self.seq1(s_next).to('cuda:1')

        s_prev = self.seq2(s_prev)
        ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

        return torch.cat(ret)

setup = "model = PipelineParallelResNet50()"
pp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
pp_mean, pp_std = np.mean(pp_run_times), np.std(pp_run_times)

plot([mp_mean, rn_mean, pp_mean],
     [mp_std, rn_std, pp_std],
     ['Model Parallel', 'Single GPU', 'Pipelining Model Parallel'],
     'mp_vs_rn_vs_pp.png')Copy
请注意，设备到设备的张量复制操作在源设备和目标设备上的当前流上同步。 如果创建多个流，则必须确保复制操作正确同步。 在完成复制操作之前写入源张量或读取/写入目标张量可能导致不确定的行为。 上面的实现仅在源设备和目标设备上都使用默认流，因此不必强制执行其他同步。



实验结果表明，对并行 ResNet50 进行建模的流水线输入可大致加快3.75/2.51-1=49%的速度，加快训练过程。 距离理想的 100% 加速仍然相去甚远。 由于我们在管道并行实现中引入了新参数split_sizes，因此尚不清楚新参数如何影响整体训练时间。 直观地讲，使用较小的split_size会导致许多小的 CUDA 内核启动，而使用较大的split_size会导致在第一次和最后一次拆分期间出现较长的空闲时间。 两者都不是最优的。 对于此特定实验，可能会有最佳的split_size配置。 让我们尝试通过使用几个不同的split_size值进行实验来找到它。

means = []
stds = []
split_sizes = [1, 3, 5, 8, 10, 12, 20, 40, 60]

for split_size in split_sizes:
    setup = "model = PipelineParallelResNet50(split_size=%d)" % split_size
    pp_run_times = timeit.repeat(
        stmt, setup, number=1, repeat=num_repeat, globals=globals())
    means.append(np.mean(pp_run_times))
    stds.append(np.std(pp_run_times))

fig, ax = plt.subplots()
ax.plot(split_sizes, means)
ax.errorbar(split_sizes, means, yerr=stds, ecolor='red', fmt='ro')
ax.set_ylabel('ResNet50 Execution Time (Second)')
ax.set_xlabel('Pipeline Split Size')
ax.set_xticks(split_sizes)
ax.yaxis.grid(True)
plt.tight_layout()
plt.savefig("split_size_tradeoff.png")
plt.close(fig)Copy


结果表明，将split_size设置为 12 可获得最快的训练速度，从而导致3.75/2.43-1=54%加速。 仍有机会进一步加快训练过程。 例如，对cuda:0的所有操作都放在其默认流上。 这意味着下一个拆分的计算不能与上一个拆分的复制操作重叠。 但是，由于上一个和下一个分割是不同的张量，因此将一个计算与另一个副本重叠是没有问题的。 实现需要在两个 GPU 上使用多个流，并且不同的子网结构需要不同的流管理策略。 由于没有通用的多流解决方案适用于所有模型并行用例，因此在本教程中将不再讨论。

注意：

这篇文章显示了几个性能指标。 当您在自己的计算机上运行相同的代码时，您可能会看到不同的数字，因为结果取决于底层的硬件和软件。 为了使您的环境获得最佳性能，一种正确的方法是首先生成曲线以找出最佳分割尺寸，然后将该分割尺寸用于管道输入。

脚本的总运行时间：（6 分钟 20.515 秒）

下载 Python 源码：model_parallel_tutorial.py

下载 Jupyter 笔记本：model_parallel_tutorial.ipynb

由 Sphinx 画廊生成的画廊

我们一直在努力

apachecn/pytorch-doc-zh

   ML | ApacheCN

Copyright © ibooker.org.cn 2019 all right reserved，由 ApacheCN 团队提供支持该文件修订时间： 2021-03-16 10:00:12



输入并搜索
PyTorch 中文官方教程 1.7
学习 PyTorch
PyTorch 深度学习：60 分钟的突击
通过示例学习 PyTorch
torch.nn到底是什么？
使用 TensorBoard 可视化模型，数据和训练
图片/视频
音频
文本
强化学习
在生产中部署 PyTorch 模型
前端 API
模型优化
并行和分布式训练
PyTorch 分布式概述
单机模型并行最佳实践
分布式数据并行入门
用 PyTorch 编写分布式应用
分布式 RPC 框架入门
使用分布式 RPC 框架实现参数服务器
使用 RPC 的分布式管道并行化
使用异步执行实现批量 RPC 处理
将分布式DataParallel与分布式 RPC 框架相结合
 编辑本页
 -
分布式数据并行入门
分布式数据并行入门
原文：https://pytorch.org/tutorials/intermediate/ddp_tutorial.html

作者：Shen Li

编辑：Joe Zhu

先决条件：

PyTorch 分布式概述
DistributedDataParallel API 文档
DistributedDataParallel注意事项
DistributedDataParallel（DDP）在模块级别实现可在多台计算机上运行的数据并行性。 使用 DDP 的应用应产生多个进程，并为每个进程创建一个 DDP 实例。 DDP 在torch.distributed包中使用集体通信来同步梯度和缓冲区。 更具体地说，DDP 为model.parameters()给定的每个参数注册一个 Autograd 挂钩，当在后向传递中计算相应的梯度时，挂钩将触发。 然后，DDP 使用该信号触发跨进程的梯度同步。 有关更多详细信息，请参考 DDP 设计说明。

推荐的使用 DDP 的方法是为每个模型副本生成一个进程，其中一个模型副本可以跨越多个设备。 DDP 进程可以放在同一台计算机上，也可以在多台计算机上，但是 GPU 设备不能在多个进程之间共享。 本教程从一个基本的 DDP 用例开始，然后演示了更高级的用例，包括检查点模型以及将 DDP 与模型并行结合。

注意

本教程中的代码在 8-GPU 服务器上运行，但可以轻松地推广到其他环境。

DataParallel和DistributedDataParallel之间的比较
在深入探讨之前，让我们澄清一下为什么尽管增加了复杂性，但还是考虑使用DistributedDataParallel而不是DataParallel：

首先，DataParallel是单进程，多线程，并且只能在单台机器上运行，而DistributedDataParallel是多进程，并且适用于单机和多机训练。 即使在单台机器上，DataParallel通常也比DistributedDataParallel慢，这是因为跨线程的 GIL 争用，每次迭代复制的模型以及分散输入和收集输出所带来的额外开销。
回顾先前的教程，如果模型太大而无法容纳在单个 GPU 上，则必须使用模型并行将其拆分到多个 GPU 中。 DistributedDataParallel与模型并行一起使用； DataParallel目前没有。 当 DDP 与模型并行组合时，每个 DDP 进程将并行使用模型，而所有进程共同将并行使用数据。
如果您的模型需要跨越多台机器，或者您的用例不适合数据并行性范式，请参阅 RPC API ，以获得更多通用的分布式训练支持。
基本用例
要创建 DDP 模块，请首先正确设置过程组。 更多细节可以在用 PyTorch 编写分布式应用中找到。

import os
import sys
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP

def setup(rank, world_size):
    if sys.platform == 'win32':
        # Distributed package only covers collective communications with Gloo
        # backend and FileStore on Windows platform. Set init_method parameter
        # in init_process_group to a local file.
        # Example init_method="file:///f:/libtmp/some_file"
        init_method="file:///{your local file path}"

        # initialize the process group
        dist.init_process_group(
            "gloo",
            init_method=init_method,
            rank=rank,
            world_size=world_size
        )
    else:
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '12355'

        # initialize the process group
        dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()Copy
现在，让我们创建一个玩具模块，将其与 DDP 封装在一起，并提供一些虚拟输入数据。 请注意，由于 DDP 会将模型状态从等级 0 进程广播到 DDP 构造器中的所有其他进程，因此您不必担心不同的 DDP 进程从不同的模型参数初始值开始。

class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(10, 5)

    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))

def demo_basic(rank, world_size):
    print(f"Running basic DDP example on rank {rank}.")
    setup(rank, world_size)

    # create model and move it to GPU with id rank
    model = ToyModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()

def run_demo(demo_fn, world_size):
    mp.spawn(demo_fn,
             args=(world_size,),
             nprocs=world_size,
             join=True)Copy
如您所见，DDP 包装了较低级别的分布式通信详细信息，并提供了干净的 API，就好像它是本地模型一样。 梯度同步通信发生在反向传递过程中，并且与反向计算重叠。 当backward()返回时，param.grad已经包含同步梯度张量。 对于基本用例，DDP 仅需要几个 LoC 即可设置流程组。 在将 DDP 应用到更高级的用例时，需要注意一些警告。

带偏差的处理速度
在 DDP 中，构造器，正向传播和反向传递都是分布式同步点。 预期不同的进程将启动相同数量的同步，并以相同的顺序到达这些同步点，并在大致相同的时间进入每个同步点。 否则，快速流程可能会提早到达，并在等待流浪者时超时。 因此，用户负责平衡流程之间的工作负载分配。 有时，由于例如网络延迟，资源争夺，不可预测的工作量峰值，不可避免地会出现处理速度偏差。 为了避免在这种情况下超时，请在调用init_process_group时传递足够大的timeout值。

保存和加载检查点
在训练过程中通常使用torch.save和torch.load来检查点模块并从检查点中恢复。 有关更多详细信息，请参见保存和加载模型。 使用 DDP 时，一种优化方法是仅在一个进程中保存模型，然后将其加载到所有进程中，从而减少写开销。 这是正确的，因为所有过程都从相同的参数开始，并且梯度在反向传播中同步，因此优化程序应将参数设置为相同的值。 如果使用此优化，请确保在保存完成之前不要启动所有进程。 此外，在加载模块时，您需要提供适当的map_location参数，以防止进程进入其他设备。 如果缺少map_location，则torch.load将首先将模块加载到 CPU，然后将每个参数复制到保存位置，这将导致同一台机器上的所有进程使用相同的设备集。 有关更高级的故障恢复和弹性支持，请参考这里。

def demo_checkpoint(rank, world_size):
    print(f"Running DDP checkpoint example on rank {rank}.")
    setup(rank, world_size)

    model = ToyModel().to(rank)
    ddp_model = DDP(model, device_ids=[rank])

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)

    CHECKPOINT_PATH = tempfile.gettempdir() + "/model.checkpoint"
    if rank == 0:
        # All processes should see same parameters as they all start from same
        # random parameters and gradients are synchronized in backward passes.
        # Therefore, saving it in one process is sufficient.
        torch.save(ddp_model.state_dict(), CHECKPOINT_PATH)

    # Use a barrier() to make sure that process 1 loads the model after process
    # 0 saves it.
    dist.barrier()
    # configure map_location properly
    map_location = {'cuda:%d' % 0: 'cuda:%d' % rank}
    ddp_model.load_state_dict(
        torch.load(CHECKPOINT_PATH, map_location=map_location))

    optimizer.zero_grad()
    outputs = ddp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(rank)
    loss_fn = nn.MSELoss()
    loss_fn(outputs, labels).backward()
    optimizer.step()

    # Not necessary to use a dist.barrier() to guard the file deletion below
    # as the AllReduce ops in the backward pass of DDP already served as
    # a synchronization.

    if rank == 0:
        os.remove(CHECKPOINT_PATH)

    cleanup()Copy
将 DDP 与模型并行性结合起来
DDP 还可以与多 GPU 模型一起使用。 当训练具有大量数据的大型模型时，DDP 包装多 GPU 模型特别有用。

class ToyMpModel(nn.Module):
    def __init__(self, dev0, dev1):
        super(ToyMpModel, self).__init__()
        self.dev0 = dev0
        self.dev1 = dev1
        self.net1 = torch.nn.Linear(10, 10).to(dev0)
        self.relu = torch.nn.ReLU()
        self.net2 = torch.nn.Linear(10, 5).to(dev1)

    def forward(self, x):
        x = x.to(self.dev0)
        x = self.relu(self.net1(x))
        x = x.to(self.dev1)
        return self.net2(x)Copy
将多 GPU 模型传递给 DDP 时，不得设置device_ids和output_device。 输入和输出数据将通过应用或模型forward()方法放置在适当的设备中。

def demo_model_parallel(rank, world_size):
    print(f"Running DDP with model parallel example on rank {rank}.")
    setup(rank, world_size)

    # setup mp_model and devices for this process
    dev0 = rank * 2
    dev1 = rank * 2 + 1
    mp_model = ToyMpModel(dev0, dev1)
    ddp_mp_model = DDP(mp_model)

    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_mp_model.parameters(), lr=0.001)

    optimizer.zero_grad()
    # outputs will be on dev1
    outputs = ddp_mp_model(torch.randn(20, 10))
    labels = torch.randn(20, 5).to(dev1)
    loss_fn(outputs, labels).backward()
    optimizer.step()

    cleanup()

if __name__ == "__main__":
    n_gpus = torch.cuda.device_count()
    if n_gpus < 8:
      print(f"Requires at least 8 GPUs to run, but got {n_gpus}.")
    else:
      run_demo(demo_basic, 8)
      run_demo(demo_checkpoint, 8)
      run_demo(demo_model_parallel, 4)Copy
我们一直在努力

apachecn/pytorch-doc-zh

   ML | ApacheCN

Copyright © ibooker.org.cn 2019 all right reserved，由 ApacheCN 团队提供支持该文件修订时间： 2021-03-16 10:00:12
